In [1]:
from bs4 import BeautifulSoup
from bs4 import element
import urllib
import re
import ssl
#from selenium import webdriver
import pandas as pd
import json
import csv
from graphics import *
import time
from tqdm import tqdm


def get_soup(url):
    #url = 'https://assignment-api.uspto.gov/patent/advancedSearch?ownerName=Numerate%2C%20Inc';
    gcontext = ssl.SSLContext()
    html_page = urllib.request.urlopen(url, context=gcontext)
    soup = BeautifulSoup(html_page) #'lxml-xml'
    #print(soup.prettify())
    return soup;


def row_item(soup):
    
    if soup('str', attrs={'name':'displayId'}) != []:
        for s in soup('str', attrs={'name':'displayId'}):
            print(s.string)

        pat = s.find_parent('doc');
        pats = pat.find_previous_siblings();
        pats.insert(0,pat)
        return pats;
    else:
        return None;
    

def str_collect(apat, name, tag, sub_tag=None):   
    lstr = []
    ro_arr = apat(tag, attrs={'name':name})
    for ustr in ro_arr:
        if sub_tag == None:
            lstr.append(ustr.string)
        else:
            tag_str = ustr(sub_tag);
            for tag in tag_str:
                lstr.append(tag.text)
    return lstr;


def parent_doc(soup):
    
    if soup('str', attrs={'name':'displayId'}) != []:
        for s in soup('str', attrs={'name':'displayId'}):
            if not quiet: 
                print(s.string)
            else:
                pass

        pat = s.find_parent('doc');            # find the parent using the last s
        pats = pat.find_previous_siblings();
        pats.insert(0,pat)                     # add last s to the beginning; why?
        return pats;
    else:
        return None;
    

def get_soup2(url):
    gcontext = ssl.SSLContext()
    html_page = urllib.request.urlopen(url, context=gcontext)
    soup = BeautifulSoup(html_page, 'lxml-xml') #'lxml-xml'
    #print(soup.prettify())
    return soup;

parent_company = 'pfizer';     #editable field; name should match the .csv file

find_subsids = True;     # True if company public
find_patents = True;

save = True; quiet = False;   #if save is true then save patents; if quiet is true, don't output during run

if find_subsids:
    print('Running subsidiary finder...')

    myurl = 'https://www.sec.gov/Archives/edgar/data/78003/000007800319000015/pfe-exhibit21x12312018x10k.htm'
    #https://www.sec.gov/Archives/edgar/data/200406/000020040619000009/ex21-subsidiariesxform10xk.htm
    soup = get_soup(myurl);
    #print(soup.prettify())

    ## Find the subsidiaries ##    

    #print(soup.find('tr'))
    start_str = 'Company Name';
    count_a = 0
    count_b = 0
    start_row = element.Tag
    tr_sections = soup.find_all('tr');

    for row in tr_sections: 
        if row('td')[0].text == 'Company Name': #row.td.text
            start_row = row;
            start_index = count_b;
            count_a += 0;
        else: 
            count_a += 1;
            count_b += 1;

    subsids_name = [];
    subsids_est = [];
    #subsids_name.append(start_row('td')[0].text);
    #subsids_est.append(start_row('td')[1].text);

    for i in range(start_index+1, len(tr_sections)):
        if ((tr_sections[i]('td')[0].text != None) and (tr_sections[i]('td')[0].text != '')):
            #print(tr_sections[i]('td')[0].text, ' *type* ', type(tr_sections[i]('td')[0].text))
            subsids_name.append(tr_sections[i]('td')[0].text);
            subsids_est.append(tr_sections[i]('td')[1].text);

    s1 = pd.Series(subsids_name);
    s2 = pd.Series(subsids_est);

    df = pd.DataFrame()
    df['Subsidiary Name'] = s1;
    df['Location inc.'] = s2; # here specifing the location of the company's incorporation


    s1
    file_str = "C:/Users/APL29/Documents/CODE/Subsidiaries/" + parent_company +".csv";
    df.to_csv(file_str)

if find_patents:
    
    #save = False; quiet = False;       # set run parameters
    print('Running patent finder...')
    ## Run Code Start ##
    # import subsidiary names and locations
    
    # path to subsidiary csv file
    #parent_company = 'schrodinger';
    file_str = "C:/Users/APL29/Documents/CODE/Subsidiaries/" + parent_company +".csv";

    # simply count the subsidiaries; sanity check
    subsids_comp = [];
    with open(file_str) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0;
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are: {" ".join(row)}')
                line_count += 1
            else:
                #print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
                subsids_comp.append(row[1]);
                line_count += 1
        print(f'Processed {line_count} lines.')

    ###  Start of Patent Search ###  # given that subsidiaries have been loaded from csv file
    #save = True; quiet = True;       # set run parameters
    companies = subsids_comp;
    df = pd.DataFrame()

    # main loop for searching all applications/patents for a given 
    for c in tqdm(companies):        # Note: tqdm calls the status indicator library 
        #company_name = 'Numerate';
        company_name = urllib.parse.quote(c);    # prepares company name string which has spaces into URL format (i.e. %)
        urls = ['https://assignment-api.uspto.gov/patent/advancedSearch?ownerName=' + company_name,'https://assignment-api.uspto.gov/patent/advancedSearch?priorOwnerName=' + company_name];
        parser = 'lxml-xml';

        count = 0;
        for myurl in urls:
            #print(type(myurl))
            if not quiet: time.sleep(1) #time.sleep(delay)  
            soup = get_soup2(myurl)     # scrape api-uspto
            pats = parent_doc(soup)     # collect all patent handles

            with open('assignment_tags_oi.json', 'r') as fp:
                atags = json.load(fp)   # load the desired tags to filter from full patent scrape

            mains = {};
            patdocs = {};
            tmp = [];

            if pats != None:
                for apat in pats:
                    mains = {};
                    num_apps = len(str_collect(apat, 'applNum',atags['applNum'][0],atags['applNum'][1]))
                    for a in range(num_apps):
                        for key in atags.keys():
                            #print('key: ', key)
                            if len(atags[key]) == 1:
                                mains[key] = str_collect(apat, key,atags[key][0])[0]

                            elif len(atags[key]) == 2:
                                tmp = str_collect(apat, key, atags[key][0], atags[key][1])
                                if len(tmp) > 1 and a <= (len(tmp)-1):
                                    mains[key] = tmp[a];
                                    #print('tmp[a] = ', tmp[a])
                                else:
                                    mains[key] = tmp[0];
                                #mains[key] = str_collect(apat, key,atags[key][0],atags[key][1])
                                #print(mains[key])
                            else:
                                print('Error with the json tags');
                        patdocs[count] = mains;
                        mains = {};
                        count += 1;
                        #patdocs.append(mains);
                df_a = pd.DataFrame(patdocs);
                df_a = df_a.transpose()
                df = pd.concat([df, df_a]);    
                #df_a = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in patdocs.items() ]))
                #df_a = df_a.transpose()
                #df = pd.concat([df, df_a]);
            elif not quiet:
                print('No Patent Properties found for: ', c, ' at URL ', urls.index(myurl)+1)

        if save:
            df.to_csv("C:/Users/APL29/Documents/CODE/Patent_pulls/Pharma_subsidiaries/" + parent_company +".csv")
 

Running patent finder...
Column names are:  Subsidiary Name Location inc.
Processed 15 lines.


  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

049059-0575
047971-0357
047972-0960
047971-0324
047972-0902
048305-0181
049062-0202
048305-0129
048066-0788
049368-0445
047971-0357
047972-0960
048305-0181
048308-0520
049062-0202
049062-0247


  7%|█████▉                                                                             | 1/14 [00:05<01:11,  5.50s/it]

043728-0801
049301-0499
042755-0925
042725-0204
042027-0018
045997-0698
043890-0667
039184-0622
049208-0237
042268-0057
039874-0754
039802-0727
039803-0552
039811-0867
039812-0309
039676-0381


...

 86%|██████████████████████████████████████████████████████████████████████▎           | 12/14 [01:05<00:07,  3.83s/it]

050679-0151
049059-0846
050106-0548
050106-0251
048003-0343
050076-0814
048039-0216
050076-0001
No Patent Properties found for:  Nimbus Lakshmi, INC  at URL  2


 93%|████████████████████████████████████████████████████████████████████████████▏     | 13/14 [01:08<00:03,  3.59s/it]

049567-0552
047628-0816
049567-0250
049743-0879
049743-0906
048999-0206
047628-0717
048821-0826
048259-0004
049569-0680
049746-0129
039490-0648
039549-0550
039360-0232
049510-0756
048825-0801


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [01:11<00:00,  3.57s/it]


In [ ]:
#df = df.reset_index(drop=True)
df

In [ ]:
 df.to_csv("C:/Users/APL29/Documents/CODE/Patent_pulls/Pharma_subsidiaries/" + parent_company +".csv")

In [ ]:
import time
from tqdm import tqdm

for i in tqdm(companies):
    time.sleep(3)


In [ ]:
mains[key] = tmp;

In [ ]:
urllib.parse.quote(companies[0])

In [ ]:
urls.index(myurl)
myurl

In [ ]:
from graphics import *


def main():
    win = GraphWin('Face', 200, 150) # give title and dimensions
    #win.yUp() # make right side up coordinates!

    head = Circle(Point(40,100), 25) # set center and radius
    head.setFill("yellow")
    head.draw(win)

    eye1 = Circle(Point(30, 105), 5)
    eye1.setFill('blue')
    eye1.draw(win)

    eye2 = Line(Point(45, 105), Point(55, 105)) # set endpoints
    eye2.setWidth(3)
    eye2.draw(win)

    mouth = Oval(Point(30, 90), Point(50, 85)) # set corners of bounding box
    mouth.setFill("red")
    mouth.draw(win)

    label = Text(Point(100, 120), 'A face')
    label.draw(win)

    message = Text(Point(win.getWidth()/2, 20), 'Click anywhere to quit.')
    message.draw(win)
    pt = Point(100, 50)
    line = Line(pt, Point(150, 100))
    line.draw(win)
    win.getMouse()
    line.move(10,40)
    win.getMouse()
    win.close()

main()

In [ ]:
from graphics import *
import numpy as np

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x,y)

r = 0; p = 0; x = 0; y = 0;
width = 400; height = 300;
cx = width/2; cy = height/2;
win = GraphWin('Face', width, height) # give title and dimensions
delay = 0.05 # seconds
#skip = True;

while r < cy:
    p += 2*np.pi/18;
    if p > 2*np.pi:
        print('True')
        p = 0;
        r += 40;
    pt1 = Point(cx+x, cy+y);
    [x, y] = pol2cart(r, p)
    pt2 = Point(cx+x, cy+y);
    line = Line(pt1, pt2);
    if p != 0:
        line.draw(win)
    time.sleep(delay)
    if r > 10 and p != 0:
        cptx = pt1.x+(pt2.x-pt1.x)/2
        cpty = pt1.y+(pt2.y-pt1.y)/2
        circ = Circle(Point(cptx,cpty), 5);
        circ.draw(win)
        time.sleep(delay)

win.getMouse()
win.close()


In [ ]:
p =0; r =0;
while True:
    p += 2*np.pi/18
    print(p, p-2*np.pi)
    if p == 2*np.pi:
        print(p)


In [ ]:
r = 0; p = 0; x = 0; y = 0;
width = 400; height = 300;
cx = width/2; cy = height/2;
win = GraphWin('Face', width, height) # give title and dimensions

test_pt = line.getP1;
cptx = pt1.x+(pt2.x-pt1.x)/2
cpty = pt1.y+(pt2.y-pt1.y)/2
circ = Circle(Point(cptx,cpty), 5);
circ.draw(win)

win.getMouse()
win.close()
#open second window and drop down to subspace window upon mouse click to view
#circle, clicking the whitespace to activate and then close the window. 


In [ ]:
pt1.x+(pt2.x-pt1.x)/2